In [35]:
from langchain_groq import ChatGroq

In [36]:
from langchain_groq import ChatGroq

llm = ChatGroq(
    temperature=0,  
    groq_api_key='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',  
    model="llama-3.1-70b-versatile"
)

response = llm.invoke("First female nobel prize winner")

print(response.content)

The first female Nobel Prize winner was Marie Curie. She won the Nobel Prize in Physics in 1903, along with her husband Pierre Curie and Henri Becquerel, for their pioneering work on radioactivity.


In [37]:
from langchain.document_loaders import WebBaseLoader

loader = WebBaseLoader("https://jobs.apple.com/en-ie/details/200575619/internship-software-engineer-intern?team=STDNT")
page_data = loader.load().pop().page_content

print(page_data)











Internship - Software Engineer Intern - Jobs at Apple (IE)







AppleStoreMaciPadiPhone WatchAirPodsTV & HomeEntertainmentAccessoriesSupport


0+
Jobs at AppleOpen MenuClose Menu

      Work at Apple
    
 

      Life at Apple
    
 

      Profile
    
 

      Sign In
    
 
Search
Jobs at Apple
Internship - Software Engineer InternCork, County Cork, IrelandStudentsAdd to Favourites Internship - Software Engineer InternRemoved from favouritesAdd a favoriteCloseTo view your favourites, sign in with your Apple Account.Sign InDon’t have an Apple Account?Create one nowForgot your Apple Account or password?Submit CVInternship - Software Engineer InternBack to search resultsSummaryPosted: 12 Nov 2024Weekly Hours: 39 Role Number:200575619Our Software Engineering team is a passionate group dedicated to improving Apple software for our international users. We are looking for a highly motivated engineer with strong software development, interpersonal, and language skills.
Descripti

In [38]:
from langchain.prompts import PromptTemplate

prompt_extract = PromptTemplate.from_template(
    """
    ### SCRAPED TEXT:
    {page_data}
    ### INSTRUCTION
    The scraped text is from the career's page of a website.
    Your job is to extract the job postings and return them in JSON format containing the following keys:
    'role', 'experience', 'skills', and 'description'.
    Only return the valid JSON.
    ### VALID JSON (NO PREAMBLE)
    """
)

chain_extract = prompt_extract | llm 
res = chain_extract.invoke(input={'page_data':page_data})
print(res.content)

```json
{
  "role": "Software Engineer Intern",
  "experience": "Pursuing a degree in Computer Science or equivalent experience",
  "skills": [
    "Apple development APIs (UIKit, Cocoa, Foundation)",
    "Building software for Apple platforms (watchOS, tvOS, iOS, macOS)",
    "Software testing and version control",
    "Swift programming language",
    "Scripting languages (e.g. Python, Ruby, Perl)",
    "Computational/Corpus Linguistics",
    "International text issues such as right-to-left layout and Unicode"
  ],
  "description": "You will work with a wide array of Apple software engineering teams, translators and project managers to ensure that Apple delivers world-class software products to our international markets. Your role is to make sure Apple software products work beautifully all over the world."
}
```


In [39]:
from langchain_core.output_parsers import JsonOutputParser

json_parser = JsonOutputParser()
json_res = json_parser.parse(res.content)
json_res

{'role': 'Software Engineer Intern',
 'experience': 'Pursuing a degree in Computer Science or equivalent experience',
 'skills': ['Apple development APIs (UIKit, Cocoa, Foundation)',
  'Building software for Apple platforms (watchOS, tvOS, iOS, macOS)',
  'Software testing and version control',
  'Swift programming language',
  'Scripting languages (e.g. Python, Ruby, Perl)',
  'Computational/Corpus Linguistics',
  'International text issues such as right-to-left layout and Unicode'],
 'description': 'You will work with a wide array of Apple software engineering teams, translators and project managers to ensure that Apple delivers world-class software products to our international markets. Your role is to make sure Apple software products work beautifully all over the world.'}

In [40]:
type(json_res)

dict

In [57]:
import pandas as pd

df = pd.read_csv('/Users/vasukhare/Desktop/Cold Email using llama 3.1/Github.csv')
print(df)

                                           Techstack  \
0  Machine learning, Python, Tensorflow, Tableu, ...   

                        Links  
0  https://github.com/Vasuk12  


In [81]:
import chromadb
import uuid

client = chromadb.PersistentClient('vectorstore')
collection = client.get_or_create_collection(name="portfolio")

if not collection.count():
    for _, row in df.iterrows():
        collection.add(documents=row["Techstack"],
                       metadatas={"link":row["Links"]},
                       ids = [str(uuid.uuid4())])





OperationalError: attempt to write a readonly database

In [78]:
links = collection.query(query_texts=["Experiance with Python"], n_results=1)
links

{'ids': [['2e911c4f-f6cf-4aba-a84e-cd49d143a4d8']],
 'embeddings': None,
 'documents': [['Machine Learning, Python, TensorFlow']],
 'uris': None,
 'data': None,
 'metadatas': [[{'link': 'https://example.com/ml-python-portfolio'}]],
 'distances': [[1.0907106660946801]],
 'included': [<IncludeEnum.distances: 'distances'>,
  <IncludeEnum.documents: 'documents'>,
  <IncludeEnum.metadatas: 'metadatas'>]}

In [44]:
job = json_res
job['skills']

['Apple development APIs (UIKit, Cocoa, Foundation)',
 'Building software for Apple platforms (watchOS, tvOS, iOS, macOS)',
 'Software testing and version control',
 'Swift programming language',
 'Scripting languages (e.g. Python, Ruby, Perl)',
 'Computational/Corpus Linguistics',
 'International text issues such as right-to-left layout and Unicode']

In [45]:
job

{'role': 'Software Engineer Intern',
 'experience': 'Pursuing a degree in Computer Science or equivalent experience',
 'skills': ['Apple development APIs (UIKit, Cocoa, Foundation)',
  'Building software for Apple platforms (watchOS, tvOS, iOS, macOS)',
  'Software testing and version control',
  'Swift programming language',
  'Scripting languages (e.g. Python, Ruby, Perl)',
  'Computational/Corpus Linguistics',
  'International text issues such as right-to-left layout and Unicode'],
 'description': 'You will work with a wide array of Apple software engineering teams, translators and project managers to ensure that Apple delivers world-class software products to our international markets. Your role is to make sure Apple software products work beautifully all over the world.'}

In [46]:
prompt_email = PromptTemplate.from_template(
        """
        ### JOB DESCRIPTION:
        {job_description}
        
        ### INSTRUCTION:
        You are Vasu, a student at the National College of Ireland doing your Master’s in Artificial Intelligence.
        You are seeking opportunities for internships or jobs where you can leverage your skills in AI, machine learning, and software development.
        Your expertise includes building predictive models, natural language processing, data analysis, and software development. 
        You have worked on projects such as an Academic-Performance-Predictor, a Weather-Corona-Chatbot, and a Cold Email Generator that you developed using Llama 3.1. 
        Your goal is to connect with companies like Apple to explore how your skills and projects can contribute to their innovative work.
        Also add the github link to showcase Apple ur portfolio: {link_list}
        ### EMAIL (NO PREAMBLE):
        
        """
        )

chain_email = prompt_email | llm
res = chain_email.invoke({"job_description": str(job), "link_list": links})
print(res.content)

Subject: Application for Software Engineer Intern at Apple

Dear Hiring Manager,

I am excited to apply for the Software Engineer Intern position at Apple, as advertised on [Job Title]. As a student at the National College of Ireland pursuing my Master's in Artificial Intelligence, I am confident that my skills and experience make me an ideal candidate for this role.

With a strong foundation in software development, machine learning, and natural language processing, I am well-equipped to contribute to Apple's innovative work. My experience in building predictive models, data analysis, and software development has prepared me to work with a wide array of Apple software engineering teams, translators, and project managers to ensure that Apple delivers world-class software products to our international markets.

I am particularly drawn to Apple's commitment to delivering high-quality software products that work beautifully all over the world. My passion for AI and machine learning aligns

In [ ]:
#  You are Mohan, a business development executive at AtliQ. AtliQ is an AI & Software Consulting company dedicated to facilitating
#         the seamless integration of business processes through automated tools. 
#         Over our experience, we have empowered numerous enterprises with tailored solutions, fostering scalability, 
#         process optimization, cost reduction, and heightened overall efficiency. 
#         Your job is to write a cold email to the client regarding the job mentioned above describing the capability of AtliQ 
#         in fulfilling their needs.
#         Also add the most relevant ones from the following links to showcase Atliq's portfolio: {link_list}
#         Remember you are Mohan, BDE at AtliQ. 